In [1]:
import sqlite3
import pandas as pd

In [18]:
follows = pd.read_csv("follows.csv")
follows

,user_id,follows,date
0,1,2,1993-09-01
1,2,1,1989-01-01
2,3,1,1993-07-01
3,2,3,1994-10-10
4,3,2,1995-03-01
5,4,2,1988-08-08
6,4,1,1988-08-08
7,1,4,1994-04-02
8,1,5,2000-01-01
9,5,1,2000-01-02


In [3]:
users = pd.read_csv("users.csv").head()
users.head()

,user_id,first_name,last_name,house
0,1,Harry,Potter,Gryffindor
1,2,Ron,Wesley,Gryffindor
2,3,Hermonie,Granger,Gryffindor
3,4,Ginny,Weasley,Gryffindor
4,5,Draco,Malfoy,Slytherin


# Convert CSV to DataFrame to SQL_table

In [4]:
# Connecting to sqlite
conn = sqlite3.connect("C:\sqlite\db\pythonsqlite.db")
#Creating a cursor object using the cursor() method
cursor = conn.cursor()
# Convert cdv to df to sql table
pd.read_csv("follows.csv").to_sql('table_follows', conn, if_exists='append', index=False)
pd.read_csv('users.csv').to_sql('table_users', conn, if_exists='append', index=False)

Check if it works :

In [5]:
def write(query):
    cursor.execute(query)
    rows = cursor.fetchall()
    return rows

In [6]:
query = """SELECT * 
            FROM table_follows 
            LIMIT 5"""
write(query)

[(1, 2, '1993-09-01'),
 (2, 1, '1989-01-01'),
 (3, 1, '1993-07-01'),
 (2, 3, '1994-10-10'),
 (3, 2, '1995-03-01')]

In [7]:
query = """SELECT * 
            FROM table_users 
            LIMIT 5"""
write(query)

[(1, 'Harry', 'Potter', 'Gryffindor'),
 (2, 'Ron', 'Wesley', 'Gryffindor'),
 (3, 'Hermonie', 'Granger', 'Gryffindor'),
 (4, 'Ginny', 'Weasley', 'Gryffindor'),
 (5, 'Draco', 'Malfoy', 'Slytherin')]

1. How many users are there in each house?

In [8]:
query = """SELECT house, 
                    COUNT(DISTINCT user_id) 
                    FROM table_users 
                    GROUP BY house """
write(query)

[('Gryffindor', 4), ('Hufflepuff', 1), ('Ravenclaw', 2), ('Slytherin', 2)]

2. List all following links that were created before September 1st, 1993

In [9]:
query = """SELECT DISTINCT(user_id+'_'+follows) AS link, 
                    user_id, 
                    follows, 
                    STRFTIME('%Y-%m-%d', DATE(table_follows.date)) AS period 
                    FROM table_follows WHERE period<'1993-09-01';"""
write(query)

[(3, 2, 1, '1989-01-01'),
 (4, 3, 1, '1993-07-01'),
 (6, 4, 2, '1988-08-08'),
 (5, 4, 1, '1988-08-08'),
 (11, 5, 6, '1986-01-10'),
 (8, 7, 1, '1990-02-02')]

3. List all the following links established before September 1st 1993, but this time use the users first names

Here we must create a new table containing informations about follows

In [10]:
def create(query):
    cursor.execute(query)
    return conn.commit()

In [12]:
# query = """CREATE TABLE info_follows AS SELECT DISTINCT(table_follows.follows), 
#                                         table_users.first_name, 
#                                         table_users.last_name, 
#                                         table_users.house 
#                                         FROM table_follows 
#                                         LEFT JOIN table_users ON table_users.user_id=table_follows.follows;"""
# create(query)

Check if  it works :

In [28]:
query = """SELECT *
            FROM info_follows 
            LIMIT 5"""
write(query)

[(2, 'Ron', 'Wesley', 'Gryffindor'),
 (1, 'Harry', 'Potter', 'Gryffindor'),
 (3, 'Hermonie', 'Granger', 'Gryffindor'),
 (4, 'Ginny', 'Weasley', 'Gryffindor'),
 (5, 'Draco', 'Malfoy', 'Slytherin')]

In [14]:
query = """SELECT DISTINCT(table_users.first_name+'_'+info_follows.first_name) AS link,  
                    table_users.first_name, 
                    info_follows.first_name,
                    STRFTIME('%Y-%m-%d', DATE(table_follows.date)) AS period 
                    FROM table_users 
                    LEFT JOIN table_follows ON table_users.user_id=table_follows.user_id 
                    LEFT JOIN info_follows ON table_follows.follows=info_follows.follows
                    WHERE period<'1993-09-01'; """
write(query)

[(0, 'Ron', 'Harry', '1989-01-01'),
 (0, 'Hermonie', 'Harry', '1993-07-01'),
 (0, 'Ginny', 'Harry', '1988-08-08'),
 (0, 'Ginny', 'Ron', '1988-08-08'),
 (0, 'Draco', 'Tom', '1986-01-10'),
 (0, 'Luna', 'Harry', '1990-02-02')]

4. Give a count of how many people followed each user as of 1999-12-31. Give the result in term of "users full
name, number of followers


In [23]:
query = """SELECT DISTINCT(table_users.first_name+'_'+info_follows.first_name) AS link,  
                    info_follows.first_name, 
                    COUNT(DISTINCT(table_users.first_name))
                    FROM table_users 
                    LEFT JOIN table_follows ON table_users.user_id=table_follows.user_id 
                    LEFT JOIN info_follows ON table_follows.follows=info_follows.follows
                    GROUP BY info_follows.first_name
                    HAVING STRFTIME('%Y-%m-%d', DATE(table_follows.date))<='1999-12-31'; """
write(query)

[(0, 'Cedric', 3),
 (0, 'Cho', 3),
 (0, 'Ginny', 1),
 (0, 'Harry', 6),
 (0, 'Hermonie', 1),
 (0, 'Luna', 1),
 (0, 'Ron', 3),
 (0, 'Tom', 1)]

5. List the number of users each user follows


In [22]:
query = """SELECT DISTINCT(table_users.first_name+'_'+info_follows.first_name) AS link,  
                    table_users.first_name, 
                    COUNT(DISTINCT(info_follows.first_name))
                    FROM table_users 
                    LEFT JOIN table_follows ON table_users.user_id=table_follows.user_id 
                    LEFT JOIN info_follows ON table_follows.follows=info_follows.follows
                    GROUP BY table_users.first_name; """
write(query)

[(0, 'Cedric', 1),
 (0, 'Cho', 2),
 (0, 'Draco', 2),
 (0, 'Ginny', 3),
 (0, 'Harry', 5),
 (0, 'Hermonie', 3),
 (0, 'Luna', 2),
 (0, 'Ron', 2),
 (None, 'Tom', 0)]

6. List all rows from follows where someone from one house follows someone from a different house. Please
give the result with user names.


In [31]:
query = """SELECT DISTINCT(table_follows.user_id+'_'+table_follows.follows) AS link, 
                    table_users.first_name,
                    table_users.house,
                    info_follows.first_name,
                    info_follows.house
                    FROM table_users 
                    LEFT JOIN table_follows ON table_users.user_id=table_follows.user_id
                    LEFT JOIN info_follows ON table_follows.follows=info_follows.follows
                    WHERE table_users.house!=info_follows.house; """
write(query)

[(6, 'Harry', 'Gryffindor', 'Draco', 'Slytherin'),
 (8, 'Harry', 'Gryffindor', 'Luna', 'Ravenclaw'),
 (9, 'Harry', 'Gryffindor', 'Cho', 'Ravenclaw'),
 (12, 'Hermonie', 'Gryffindor', 'Cedric', 'Hufflepuff'),
 (13, 'Ginny', 'Gryffindor', 'Cedric', 'Hufflepuff'),
 (6, 'Draco', 'Slytherin', 'Harry', 'Gryffindor'),
 (8, 'Luna', 'Ravenclaw', 'Harry', 'Gryffindor'),
 (9, 'Cho', 'Ravenclaw', 'Harry', 'Gryffindor'),
 (17, 'Cho', 'Ravenclaw', 'Cedric', 'Hufflepuff'),
 (17, 'Cedric', 'Hufflepuff', 'Cho', 'Ravenclaw')]

7. List all unrequited followings (i.e. where A follows B but B does not follow A)


In [37]:
query = """SELECT DISTINCT(table_follows.user_id+'_'+table_follows.follows) AS link, 
                    table_users.first_name,
                    table_users.house,
                    info_follows.first_name,
                    info_follows.house
                    FROM table_users 
                    LEFT JOIN table_follows ON table_users.user_id=table_follows.user_id
                    LEFT JOIN info_follows ON table_follows.follows=info_follows.follows
                    ORDER BY link
                    ; """
write(query)

[(None, 'Tom', 'Slytherin', None, None),
 (3, 'Harry', 'Gryffindor', 'Ron', 'Gryffindor'),
 (3, 'Ron', 'Gryffindor', 'Harry', 'Gryffindor'),
 (4, 'Hermonie', 'Gryffindor', 'Harry', 'Gryffindor'),
 (5, 'Harry', 'Gryffindor', 'Ginny', 'Gryffindor'),
 (5, 'Ron', 'Gryffindor', 'Hermonie', 'Gryffindor'),
 (5, 'Hermonie', 'Gryffindor', 'Ron', 'Gryffindor'),
 (5, 'Ginny', 'Gryffindor', 'Harry', 'Gryffindor'),
 (6, 'Harry', 'Gryffindor', 'Draco', 'Slytherin'),
 (6, 'Ginny', 'Gryffindor', 'Ron', 'Gryffindor'),
 (6, 'Draco', 'Slytherin', 'Harry', 'Gryffindor'),
 (8, 'Harry', 'Gryffindor', 'Luna', 'Ravenclaw'),
 (8, 'Luna', 'Ravenclaw', 'Harry', 'Gryffindor'),
 (9, 'Harry', 'Gryffindor', 'Cho', 'Ravenclaw'),
 (9, 'Cho', 'Ravenclaw', 'Harry', 'Gryffindor'),
 (11, 'Draco', 'Slytherin', 'Tom', 'Slytherin'),
 (12, 'Hermonie', 'Gryffindor', 'Cedric', 'Hufflepuff'),
 (13, 'Ginny', 'Gryffindor', 'Cedric', 'Hufflepuff'),
 (15, 'Luna', 'Ravenclaw', 'Cho', 'Ravenclaw'),
 (17, 'Cho', 'Ravenclaw', 'Cedric', 